## 1. Graph Definition

In [1]:
class Vertex(object):
    def __init__(self, key):
        self.key = key
        self.conns = {}
    
    def addConnection(self, key, weight):
        self.conns[key] = weight

class Graph(object): # weighted + directed
    def __init__(self):
        self.vertices = []
    
    def get(self, key):
        res = [v for v in self.vertices if v.key == key]
        if len(res) == 0:
            return None
        return res[0]
    
    def addVertex(self, key):
        self.vertices.append(Vertex(key))

    def addEdge(self, keySrc, keyDst, weight=1):
        if self.get(keySrc) == None:
            self.addVertex(keySrc)
        if self.get(keyDst) == None:
            self.addVertex(keyDst)
        self.get(keySrc).addConnection(keyDst, weight)

## 2. Graph Traversals
### 2.1 Depth First Search (DFS)

In [2]:
def DFS(graph:Graph, start=0):
    def step(key, res:list, visited:list):
        res.append(key)
        visited.append(key)
        for k in graph.get(key).conns.keys():
            if k not in visited:
                step(k, res, visited)
    res, visited = [], []
    step(start, res, visited)
    return res

In [3]:
graph = Graph()  
graph.addEdge(0, 1)
graph.addEdge(0, 2)
graph.addEdge(1, 2)
graph.addEdge(2, 0)
graph.addEdge(2, 3)
graph.addEdge(3, 3)
print(DFS(graph, 2))

[2, 0, 1, 3]


### 2.2 Breadth First Search (BFS)

In [4]:
def BFS(graph:Graph, start=0):
    from collections import deque
    res, visited = [], []
    queue = deque()
    queue.append(start)
    visited.append(start)
    while len(queue) > 0:
        key = queue.popleft()
        res.append(key)
        for k in graph.get(key).conns.keys():
            if k not in visited:
                queue.append(k)
                visited.append(k)
    return res

In [5]:
graph = Graph()  
graph.addEdge(0, 1)
graph.addEdge(0, 2)
graph.addEdge(1, 2)
graph.addEdge(2, 0)
graph.addEdge(2, 3)
graph.addEdge(3, 3)
print(BFS(graph, 2))

[2, 0, 3, 1]


### 3.3 Topological Sort
A topological ordering is not possible if the graph has a cycle.

In [6]:
def topSort(graph:Graph):
    def getIndegrees(indegrees):
        for v in graph.vertices:
            indegrees[v.key] = 0
        for v in graph.vertices:
            for k in v.conns.keys():
                indegrees[k] += 1
    
    from collections import deque
    res = []
    queue = deque()
    # get the indegree of each vertex
    indegrees = {}
    getIndegrees(indegrees)
    # find the vertices with indegree 0
    for v in graph.vertices:
        if indegrees[v.key] == 0:
            queue.append(v.key)
    while len(queue) > 0:
        # remove the 0-vertex
        tmp = queue.popleft()
        res.append(tmp)
        # update the neighbors then push the new 0-vertices
        for k in graph.get(tmp).conns.keys():
            indegrees[k] -= 1
            if indegrees[k] == 0:
                queue.append(k)
    if len(res) != len(graph.vertices):
        return None
    return res

In [7]:
graph = Graph()
graph.addEdge(1, 2)
graph.addEdge(1, 3)
graph.addEdge(1, 4)
graph.addEdge(2, 4)
graph.addEdge(2, 5)
graph.addEdge(3, 6)
graph.addEdge(4, 3)
graph.addEdge(4, 6)
graph.addEdge(4, 7)
graph.addEdge(5, 4)
graph.addEdge(5, 7)
graph.addEdge(7, 6)
print(topSort(graph))

[1, 2, 5, 4, 3, 7, 6]


## 3. Shortest-Path Algorithms
### 3.1 Breadth First Search (Unweighted)

### 3.2 Dijkstra’s Algorithm (Weighted)

In [8]:
def dijkstra(graph:Graph, start=0):
    record = {}
    for v in graph.vertices:
        record[v.key] = {'dist': float('inf'), 'done': 0, 'paths': [[]]}
    record[start]['dist'] = 0
    while True:
        tmp = [k for k in record.keys() if record[k]['dist'] == min([val['dist'] for val in record.values()])]
        if len(tmp) == 0:
            break
        key = tmp[0]
        record[key]['done'] = 1
        for path in record[key]['paths']:
            path.append(key)
        for vertex, weight in graph.get(key).conns:
            if record[vertex.key]['done'] == 0:
                currDist = record[vertex.key]['dist']
                newDist = record[key]['dist'] + weight
                if newDist < currDist:
                    record[vertex.key]['dist'] = newDist
                    record[vertex.key]['path'] = [_ for _ in record[key]['path']]
                elif newDist == currDist:
                    record[vertex.key]['path'].append([_ for _ in record[key]['path']])
    return record

### 3.3 Greedy Best First Search
May not find the shortest path !

https://www.redblobgames.com/pathfinding/a-star/introduction.html

In [9]:
def greedyBestFirstSearch(graph, start, goal):
    from queue import PriorityQueue
    def heuristic(pos1, pos2):
        # Manhattan distance on a square grid
        return abs(pos1[0] - pos2[0]) + abs(pos1[1] - pos2[1])
    
    def getNeighbors(pos):
        m, n = len(graph), len(graph[0])
        x, y = pos[0], pos[1]
        neighbors = []
        if y + 1 < n and graph[x][y+1] == 0:
            neighbors.append((x, y + 1))
        if x + 1 < m and graph[x+1][y] == 0:
            neighbors.append((x + 1, y))
        if y - 1 >= 0 and graph[x][y-1] == 0:
            neighbors.append((x, y - 1))
        if x - 1 >= 0 and graph[x-1][y] == 0:
            neighbors.append((x - 1, y))
        return neighbors
    
    def getPath(prev):
        path, currPos = [goal], goal
        while currPos != start:
            path.insert(0, prev[currPos])
            currPos = prev[currPos]
        return path
    
    frontier = PriorityQueue()
    frontier.put((0, start))
    prev = {}
    prev[start] = None
    while not frontier.empty():
        _, currPos = frontier.get()
        if currPos == goal:
            return getPath(prev)
        for next in getNeighbors(currPos):
            if next not in prev:
                priority = heuristic(next, goal)
                frontier.put((priority, next))
                prev[next] = currPos

graph = [[0 for _ in range(15)] for _ in range(15)]
graph[2][2:14] = [1 for _ in range(2, 14)]
graph[12][2:13] = [1 for _ in range(2, 13)]
for i in range(2, 13):
    graph[i][12] = 1
for i in range(len(graph)):
    print(graph[i])
path = greedyBestFirstSearch(graph, (12, 0), (2, 14))
print(path)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]
[0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[(12, 0), (11, 0), (10, 0), (9, 0), (8, 0), (7, 0), (6, 0), (5, 0), (4, 0), (3, 0), (2, 0), (2, 1), (1, 1), (1, 2), (1, 3), (1, 4), (1, 5), (1, 6), (1, 7), (1, 8), (1, 9), (1, 10), (1, 11), (1, 12), (1, 13), (1, 14), (2, 14)]


### 3.4 A* Search

In [10]:
def aStarSearch(graph, start, goal):
    from queue import PriorityQueue
    def heuristic(pos1, pos2):
        # Manhattan distance on a square grid
        return abs(pos1[0] - pos2[0]) + abs(pos1[1] - pos2[1])
    
    def getNeighbors(pos):
        m, n = len(graph), len(graph[0])
        x, y = pos[0], pos[1]
        neighbors = []
        if y + 1 < n and graph[x][y+1] == 0:
            neighbors.append((x, y + 1))
        if x + 1 < m and graph[x+1][y] == 0:
            neighbors.append((x + 1, y))
        if y - 1 >= 0 and graph[x][y-1] == 0:
            neighbors.append((x, y - 1))
        if x - 1 >= 0 and graph[x-1][y] == 0:
            neighbors.append((x - 1, y))
        return neighbors
    
    def getPath(prev):
        path, currPos = [goal], goal
        while currPos != start:
            path.insert(0, prev[currPos])
            currPos = prev[currPos]
        return path

    frontier = PriorityQueue()
    frontier.put((0, start))
    prev = {}
    cost = {}
    prev[start], cost[start] = None, 0
    while not frontier.empty():
        _, currPos = frontier.get()
        if currPos == goal:
            return getPath(prev)
        for next in getNeighbors(currPos):
            newCost = cost[currPos] + 1
            if next not in cost or newCost < cost[next]:
                cost[next] = newCost
                priority = newCost + heuristic(next, goal)
                frontier.put((priority, next))
                prev[next] = currPos

graph = [[0 for _ in range(15)] for _ in range(15)]
graph[2][2:14] = [1 for _ in range(2, 14)]
graph[12][2:13] = [1 for _ in range(2, 13)]
for i in range(2, 13):
    graph[i][12] = 1
for i in range(len(graph)):
    print(graph[i])
path = aStarSearch(graph, (12, 0), (2, 14))
print(path)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]
[0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[(12, 0), (11, 0), (10, 0), (9, 0), (8, 0), (7, 0), (6, 0), (5, 0), (4, 0), (3, 0), (2, 0), (2, 1), (1, 1), (1, 2), (1, 3), (1, 4), (1, 5), (1, 6), (1, 7), (1, 8), (1, 9), (1, 10), (1, 11), (1, 12), (1, 13), (1, 14), (2, 14)]
